In [2]:
!pip install flask==2.1.0

     ---------------------------------------- 0.0/95.2 kB ? eta -:--:--
     ---------------------------------------- 95.2/95.2 kB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/233.6 kB ? eta -:--:--
     ------------------------------------- 233.6/233.6 kB 14.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/96.6 kB ? eta -:--:--
     ---------------------------------------- 96.6/96.6 kB ? eta 0:00:00


In [4]:
# 모델 불러와서 확인
import joblib
import pandas as pd
# bmi_model = joblib.load('모델 경로 및 파일명',)
bmi_model = joblib.load('./bmi_model.pkl')
bmi_model

KNeighborsClassifier()

In [5]:
# 예측 진행해보기
# 학습된 상태가 그대로 보존되어 예측이 가능한 상태인지 확인
new_test = pd.DataFrame({'Height' : [185], 'Weight' : [50]})
# 예측
bmi_model.predict(new_test)
 

array(['Extremely Weak'], dtype=object)

### Flask

In [6]:
# 1. get 방식
def html_temp1():
    return """<!DOCTYPE html>
            <html>
            <head> 
            <title>Document</title>
            </head>
            <body>

            <h1>bmi를 구해보자</h1>
            <p>Height와 Weight를 입력하세요!</p>
            <form action = 'http://127.0.0.1:5000/user', method = 'GET'>
                키:<input type="text" id="Height" name="h"><br>
                몸무게:<input type="text" id="Weight" name="w">
                <input type='submit'></input>
            </form >
           
            </body>
            </html>"""

내 코드

In [17]:
# flask 코드
from flask import Flask, request, jsonify, redirect
app = Flask(__name__)

@app.route('/')
def input_data() :
    # html 페이지 실행
    return html_temp1()  # 위에서 선언한 함수를 열어줌
    

@app.route('/user' , methods = ['GET' ,'POST'])
def user() :
    if request.method == 'GET' :
        h = int(request.args.get('h')) # 사용자가 입력한 키(형변환 필요)
        w = int(request.args.get('w')) # 사용자가 입력한 몸무게
        bmi_pre = bmi_model.predict([[h, w]])  # 값을 하나씩 가져오지만 2차원으로 담아줘야지 예측을 할 수 있음
        # redirect 
        return redirect(f'http://127.0.0.1:5000?bmi_result={bmi_pre[0]}')

if __name__ == '__main__' : 
    app.run(host = '127.0.0.1' , port = '5000')


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Apr/2023 10:12:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 10:12:58] "GET / HTTP/1.1" 200 -
c:\Users\smhrd\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
127.0.0.1 - - [17/Apr/2023 10:13:01] "GET /user?h=162&w=50 HTTP/1.1" 302 -
127.0.0.1 - - [17/Apr/2023 10:13:01] "GET /?bmi_result=Normal HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 10:13:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 10:13:09] "GET /favicon.ico HTTP/1.1" 404 -
c:\Users\smhrd\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
127.0.0.1 - - [17/Apr/2023 10:13:12] "GET /user?h=162&w=50 HTTP/1.1" 302 -
127.0.0.1 - - [17/Apr/2023 10:13

In [13]:
# 2. post 방식
# Ajax 활용해서 html 구성
def html_temp2():
    return """<!DOCTYPE html>
            <html>
            <head> 
            <title>Document</title>
            <script type="text/javascript" src = "https://code.jquery.com/jquery-3.6.3.min.js"></script>
            <script>
                $(document).ready(function(){
                    $(".ajaxSend").on("click", function(){
                        var height = $('input[name=h]').val();
                        var weight = $('input[name=w]').val();
                        $.ajax({
                            url:"http://127.0.0.1:5000/user",
                            type:"POST",
                            dataType:"JSON",
                            data :{"h":height,"w":weight}
                        })
                        
                        .done(function (json){
                            $(".result").text(json.result)
                        })
                        
                        .fail(function (xhr, status, errorThrown){
                            alert("Ajax failed")
                        })
                    })
                    })
                    
            </script>
            </head>
            <body>

            <h1>bmi를 구해보자</h1>
            <p>Height와 Weight를 입력하세요!</p>
 
            키: <input type="text" id="Height" name="h"><br>
            몸무게:<input type="text" id="Weight" name="w">
            <button class='ajaxSend'>ajaxSend</button><br><br>
            <div>bmi결과: <span class="result">?</span></div>    
            
            </body>
            </html>"""


# $(".result").text(json.result)
# location.href = "https://search.naver.com/search.naver?query=" + json.result

In [16]:
# flask 코드 
from flask import Flask, request, jsonify, redirect
app = Flask(__name__) 

@app.route('/')
def input_data():
    # html 페이지 실행 
    return html_temp2()

@app.route('/user', methods =['GET','POST'])
def user():
    if request.method =='GET':
        h = request.args.get('h') # 사용자가 입력한 키
        w = request.args.get('w') # 사용자가 입력한 몸무게
        bmi_pre = bmi_model.predict([[h,w]])
        return redirect(f'http://127.0.0.1:5000?bmi_result={bmi_pre[0]}')
    else :
        # post 방식
        h = int(request.form.get('h'))
        w = int(request.form.get('w'))
        bmi_pre = bmi_model.predict([[h,w]])
        return jsonify({'result' : bmi_pre[0]})  # 비동기 방식

if __name__ == '__main__':
    app.run(host ='127.0.0.1', port = '5000')

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Apr/2023 10:09:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 10:09:06] "GET / HTTP/1.1" 200 -
c:\Users\smhrd\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
127.0.0.1 - - [17/Apr/2023 10:09:17] "POST /user HTTP/1.1" 200 -
